# __Group Project 3 - Team Terence Gorman__
<br>


### <span style = "color:blue">__Team Members: Marc Bonnot (*Team Manager*), Ce (Faye) Fang, Dojun Kim, Kimberley Fan, Jane Wang, Qianhao (Harry) Zhong__
<br>


## **1. Project Topic:**

> 
<br>


## **2. Project Goal:**

>
<br>


## **3. [A] Choice of a population, with supporting expenditure data:**

>
<br>


## **4. Data Description:**

>    
<br>


## **5. Project Deliverables:**

### *Packages Import*:

## [B] Nutritional Content of Different Foods

## [B] Nutritional Adequacy of Diet

## [A] Estimate Demand System

## [C] Counterfactual Experiments

## Visualization